<a href="https://colab.research.google.com/github/zadkel/Complex-transformer/blob/main/WNLI_%ED%86%A0%ED%81%B0%ED%99%94%2C_%EC%9D%B8%EB%8D%B1%EC%8B%B1%2C_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch
import copy
import math
import torch.nn.functional as F
import numpy as np

#d_model = 512 차원, d_ff는 2048, attention layer개수 = 8개

In [ ]:
import torchtext
from torchtext.data import get_tokenizer
import pandas as pd
tokenizer = get_tokenizer('basic_english')
WNLI_train_path = '/content/drive/MyDrive/PBL/Transformer/GLUE/GLUE-baselines/glue_data/WNLI/train.tsv' #본인 데이터 경로 위치
WNLI_dataset_train = pd.read_csv(WNLI_train_path,delimiter = '\t', on_bad_lines='skip') # 오류있는 행은 넘어가기(on_bad_lines)

WNLI_dataset_train_dropped = WNLI_dataset_train.drop(columns = ['index'])


# NAN 값 드랍
WNLI_dataset_train_dropped = WNLI_dataset_train_dropped.dropna()

WNLI_dataset_train_dropped['Tokenized_sentence1'] = WNLI_dataset_train_dropped['sentence1'].apply(tokenizer)
WNLI_dataset_train_dropped['Tokenized_sentence2'] = WNLI_dataset_train_dropped['sentence2'].apply(tokenizer)
WNLI_dataset_train_dropped = WNLI_dataset_train_dropped.drop(columns = ['sentence1'])
WNLI_dataset_train_dropped = WNLI_dataset_train_dropped.drop(columns = ['sentence2'])
WNLI_dataset_train_dropped

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


,label,Tokenized_sentence1,Tokenized_sentence2
0,1,"[i, stuck, a, pin, through, a, carrot, ., when...","[the, carrot, had, a, hole, .]"
1,1,"[john, couldn, ', t, see, the, stage, with, bi...","[john, is, so, short, .]"
2,1,"[the, police, arrested, all, of, the, gang, me...","[the, police, were, trying, to, stop, the, dru..."
3,0,"[steve, follows, fred, ', s, example, in, ever...","[steve, influences, him, hugely, .]"
4,0,"[when, tatyana, reached, the, cabin, ,, her, m...","[mother, was, careful, not, to, disturb, her, ..."
...,...,...,...
630,0,"[when, tommy, dropped, his, ice, cream, ,, tim...","[father, gave, timmy, a, sympathetic, look, .]"
631,1,"[mark, told, pete, many, lies, about, himself,...","[pete, should, have, been, more, skeptical, .]"
632,0,"[tom, said, check, to, ralph, as, he, took, hi...","[tom, said, check, to, ralph, as, he, took, to..."
633,1,"[emma, ', s, mother, had, died, long, ago, ,, ...","[emma, ', s, mother, ', s, place, had, been, t..."


In [ ]:
# 단어 사전 구축

vocab = set(word for sentence in WNLI_dataset_train_dropped['Tokenized_sentence1'] for word in sentence)
vocab.update(word for sentence in WNLI_dataset_train_dropped['Tokenized_sentence2'] for word in sentence)
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}


In [ ]:
# 임베딩 레이어 정의
embedding_dim = 128  # 임베딩 벡터의 차원
embedding_layer = nn.Embedding(len(vocab), embedding_dim)

# 모든 단어를 임베딩하여 딕셔너리에 저장
word2embedding = {}
for word, idx in word2idx.items():
    idx_tensor = torch.tensor([idx], dtype=torch.long)
    embedding_vector = embedding_layer(idx_tensor).detach().numpy().flatten()
    word2embedding[word] = embedding_vector

# get_word_embedding 함수 정의 (단일 단어에 대한 임베딩 반환)
def get_word_embedding(word, word2embedding):
    return word2embedding[word]


# 각 단어에 대한 임베딩 추가
WNLI_dataset_train_dropped['embedding_sentence1'] = WNLI_dataset_train_dropped['Tokenized_sentence1'].apply(lambda tokens: [get_word_embedding(word, word2embedding) for word in tokens])
WNLI_dataset_train_dropped['embedding_sentence2'] = WNLI_dataset_train_dropped['Tokenized_sentence2'].apply(lambda tokens: [get_word_embedding(word, word2embedding) for word in tokens])
# 결과 확인
WNLI_dataset_train_dropped

,label,Tokenized_sentence1,Tokenized_sentence2,embedding_sentence1,embedding_sentence2
0,1,"[i, stuck, a, pin, through, a, carrot, ., when...","[the, carrot, had, a, hole, .]","[[-1.1050314, 0.60642314, -0.2563558, -0.72341...","[[0.057586424, 0.4181355, 0.4533395, -0.124814..."
1,1,"[john, couldn, ', t, see, the, stage, with, bi...","[john, is, so, short, .]","[[-0.86030126, -0.40687525, -0.022677101, 1.84...","[[-0.86030126, -0.40687525, -0.022677101, 1.84..."
2,1,"[the, police, arrested, all, of, the, gang, me...","[the, police, were, trying, to, stop, the, dru...","[[0.057586424, 0.4181355, 0.4533395, -0.124814...","[[0.057586424, 0.4181355, 0.4533395, -0.124814..."
3,0,"[steve, follows, fred, ', s, example, in, ever...","[steve, influences, him, hugely, .]","[[0.78445303, 0.15688846, -2.4730175, -0.06853...","[[0.78445303, 0.15688846, -2.4730175, -0.06853..."
4,0,"[when, tatyana, reached, the, cabin, ,, her, m...","[mother, was, careful, not, to, disturb, her, ...","[[0.5429633, -1.5135834, 0.22104083, 0.4996009...","[[-0.91059184, -0.1888451, -1.5486273, -0.8618..."
...,...,...,...,...,...
630,0,"[when, tommy, dropped, his, ice, cream, ,, tim...","[father, gave, timmy, a, sympathetic, look, .]","[[0.5429633, -1.5135834, 0.22104083, 0.4996009...","[[0.23743308, 2.1000972, -2.3955066, -1.165761..."
631,1,"[mark, told, pete, many, lies, about, himself,...","[pete, should, have, been, more, skeptical, .]","[[-2.2590337, 0.46191382, -1.1606987, -0.32647...","[[0.09423723, -0.7838995, -0.85155743, 0.75307..."
632,0,"[tom, said, check, to, ralph, as, he, took, hi...","[tom, said, check, to, ralph, as, he, took, to...","[[-0.5646595, 1.4555761, -0.45129812, -0.07662...","[[-0.5646595, 1.4555761, -0.45129812, -0.07662..."
633,1,"[emma, ', s, mother, had, died, long, ago, ,, ...","[emma, ', s, mother, ', s, place, had, been, t...","[[0.53156334, -0.30170217, 1.4026098, -0.78284...","[[0.53156334, -0.30170217, 1.4026098, -0.78284..."


In [ ]:
# test data 임베딩
WNLI_test_path = '/content/drive/MyDrive/PBL/Transformer/GLUE/GLUE-baselines/glue_data/WNLI/test.tsv' #본인 데이터 경로 위치
WNLI_dataset_test = pd.read_csv(WNLI_test_path,delimiter = '\t', on_bad_lines='skip') # 오류있는 행은 넘어가기(on_bad_lines)


WNLI_dataset_test_dropped = WNLI_dataset_test.drop(columns = ['index'])

# NAN 값 드랍
WNLI_dataset_test_dropped = WNLI_dataset_test_dropped.dropna()

WNLI_dataset_test_dropped['Tokenized_sentence1'] = WNLI_dataset_test_dropped['sentence1'].apply(tokenizer)
WNLI_dataset_test_dropped['Tokenized_sentence2'] = WNLI_dataset_test_dropped['sentence2'].apply(tokenizer)
WNLI_dataset_test_dropped = WNLI_dataset_test_dropped.drop(columns = ['sentence1'])
WNLI_dataset_test_dropped = WNLI_dataset_test_dropped.drop(columns = ['sentence2'])

# 단어 사전 구축

vocab_test = set(word for sentence in WNLI_dataset_test_dropped['Tokenized_sentence1'] for word in sentence)
vocab_test.update(word for sentence in WNLI_dataset_test_dropped['Tokenized_sentence2'] for word in sentence)
word2idx_test = {word: idx for idx, word in enumerate(vocab_test)}
idx2word_test = {idx: word for word, idx in word2idx_test.items()}

# 임베딩 레이어 정의
embedding_layer_test = nn.Embedding(len(vocab_test), embedding_dim)

# 모든 단어를 임베딩하여 딕셔너리에 저장
word2embedding_test = {}
for word, idx in word2idx_test.items():
    idx_tensor_test = torch.tensor([idx], dtype=torch.long)
    embedding_vector_test = embedding_layer_test(idx_tensor_test).detach().numpy().flatten()
    word2embedding_test[word] = embedding_vector_test

# get_word_embedding 함수 정의 (단일 단어에 대한 임베딩 반환)
def get_word_embedding_test(word, word2embedding_test):
    return word2embedding_test[word]


# 각 단어에 대한 임베딩 추가
WNLI_dataset_test_dropped['embedding_sentence1'] = WNLI_dataset_test_dropped['Tokenized_sentence1'].apply(lambda tokens: [get_word_embedding_test(word, word2embedding_test) for word in tokens])
WNLI_dataset_test_dropped['embedding_sentence2'] = WNLI_dataset_test_dropped['Tokenized_sentence2'].apply(lambda tokens: [get_word_embedding_test(word, word2embedding_test) for word in tokens])
# 결과 확인
WNLI_dataset_test_dropped

,Tokenized_sentence1,Tokenized_sentence2,embedding_sentence1,embedding_sentence2
0,"[maude, and, dora, had, seen, the, trains, rus...","[horses, ran, away, when, maude, and, dora, ca...","[[-0.6909515, -1.7452266, -1.3337501, -0.33987...","[[1.3721914, 1.4835727, 1.2499837, 0.44358012,..."
1,"[maude, and, dora, had, seen, the, trains, rus...","[horses, ran, away, when, the, trains, came, i...","[[-0.6909515, -1.7452266, -1.3337501, -0.33987...","[[1.3721914, 1.4835727, 1.2499837, 0.44358012,..."
2,"[maude, and, dora, had, seen, the, trains, rus...","[horses, ran, away, when, the, puffs, came, in...","[[-0.6909515, -1.7452266, -1.3337501, -0.33987...","[[1.3721914, 1.4835727, 1.2499837, 0.44358012,..."
3,"[maude, and, dora, had, seen, the, trains, rus...","[horses, ran, away, when, the, roars, came, in...","[[-0.6909515, -1.7452266, -1.3337501, -0.33987...","[[1.3721914, 1.4835727, 1.2499837, 0.44358012,..."
4,"[maude, and, dora, had, seen, the, trains, rus...","[horses, ran, away, when, the, whistles, came,...","[[-0.6909515, -1.7452266, -1.3337501, -0.33987...","[[1.3721914, 1.4835727, 1.2499837, 0.44358012,..."
...,...,...,...,...
141,"[muriel, has, in, her, living, room, a, famous...","[it, is, very, characteristic, of, muriel, ', ...","[[-1.0802351, -0.65135366, -0.25471416, 0.6390...","[[-0.58470654, -0.9405954, 0.5628839, -1.55449..."
142,"[muriel, has, in, her, living, room, a, famous...","[it, is, very, characteristic, of, theresa, ru...","[[-1.0802351, -0.65135366, -0.25471416, 0.6390...","[[-0.58470654, -0.9405954, 0.5628839, -1.55449..."
143,"[muriel, has, in, her, living, room, a, famous...","[it, is, very, characteristic, of, cecile, d, ...","[[-1.0802351, -0.65135366, -0.25471416, 0.6390...","[[-0.58470654, -0.9405954, 0.5628839, -1.55449..."
144,"[edward, dropped, adhesive, tape, onto, his, w...","[some, of, the, glue, was, stuck, on, the, tap...","[[-0.31246555, -1.0006841, -1.3452156, -0.7380...","[[0.74742293, -0.48587713, -0.8902839, -0.4483..."
